In [2]:
import json
import pandas as pd

pd.options.mode.chained_assignment = None

In [3]:
with open('match_stats.json', 'r') as f:
    df = pd.DataFrame.from_dict(pd.json_normalize(json.loads(f.read())))

Добавим признак, который будет показывать историю личных встреч из нашего датасета. Для этого введем признак head-to-head, который будет считать сумму результатов матчей (за период до прошедшего матча) между командами по следующему принципу: +1 за победу, 0 за ничью и -1 за проигрыш.

In [4]:
teams = df['home.team'].unique()

In [5]:
def head_to_head(team1: str, team2: str, df: pd.DataFrame) -> float:
    home_matches = df[(df['home.team'] == team1) & (df['guest.team'] == team2)]
    score = 0
    score += (home_matches['home.score'] > home_matches['guest.score']).sum()
    score -= (home_matches['home.score'] < home_matches['guest.score']).sum()
    guest_matches = df[(df['home.team'] == team2) & (df['guest.team'] == team1)]
    score -= (guest_matches['home.score'] > guest_matches['guest.score']).sum()
    score += (guest_matches['home.score'] < guest_matches['guest.score']).sum()
    return score

In [6]:
df['head_to_head'] = df.apply(lambda row: head_to_head(row['home.team'], row['guest.team'], df[df['match_date'] < row['match_date']]), axis=1)

In [7]:
df.head()

,match_date,match_week,home.team,home.score,home.xG,home.possession,home.passing_accuracy,home.shots_on_target,home.yellow_cards_count,home.red_cards_count,...,guest.touches,guest.tackles,guest.interceptions,guest.aerials_won,guest.clearances,guest.offsides,guest.goal_kicks,guest.throw_ins,guest.long_balls,head_to_head
0,2022-08-05,1,Crystal Palace,0,1.2,0.56,0.84,0.20,1,0,...,599.0,29.0,9,14.0,24.0,2.0,2.0,14.0,59.0,-2
1,2022-08-06,1,Fulham,2,1.2,0.33,0.60,0.25,2,0,...,784.0,11.0,10,13.0,16.0,4.0,5.0,35.0,94.0,-1
2,2022-08-06,1,Tottenham Hotspur,4,1.5,0.58,0.83,0.44,3,0,...,554.0,14.0,13,11.0,21.0,0.0,4.0,14.0,61.0,4
3,2022-08-06,1,Newcastle United,2,1.7,0.61,0.79,0.39,0,0,...,475.0,15.0,10,16.0,37.0,0.0,12.0,26.0,70.0,0
4,2022-08-06,1,Leeds United,2,0.8,0.40,0.74,0.33,2,0,...,720.0,16.0,14,7.0,17.0,1.0,9.0,23.0,72.0,-1


### ML

Скопируем предыдущий датафрейм, чтобы можно было поменять некоторые столбцы

In [8]:
new_df = df.copy()

Напишем функцию, которая напишет результат матча (2 — победа домашней команды, 1 — ничья, 0 — победа гостей)

In [9]:
def get_result(h_score: int, g_score: int) -> int:
    if h_score > g_score:
        return 2
    elif h_score == g_score:
        return 1
    else:
        return 0

In [10]:
new_df['result'] = new_df.apply(lambda row: get_result(row['home.score'], row['guest.score']), axis = 1)

In [11]:
new_df['match_date'] = pd.to_datetime(new_df['match_date'])

In [12]:
def get_aggregate(team: str, n: int, df:pd.DataFrame) -> pd.DataFrame:
    '''
        Функция, собирающая средние показатели команды за последние n матчей в передаваемом датафрейме

        Args:
            team: Название команды
            n: Количество матчей
            df: Датафрейм с матчами

        Returns:
            df: Датафрейм (1 строчки) с агрегированными данными для команды
    '''
    df.sort_values(by = 'match_date', inplace=True, ascending=False)
    last_matches = df[(df['home.team'] == team) | (df['guest.team'] == team)][:n]
    result = pd.DataFrame(columns=['score', 'xG', 'possession',
       'passing_accuracy', 'shots_on_target',
       'yellow_cards_count', 'red_cards_count', 'fouls',
       'corners', 'crosses', 'touches', 'tackles',
       'interceptions', 'aerials_won', 'clearances',
       'offsides', 'goal_kicks', 'throw_ins',
       'long_balls'])

    # Данные списки и словари нужны для правильного переноса значений из колонок с разными названиями
    home_features = ['home.score', 'home.xG',
       'home.possession', 'home.passing_accuracy', 'home.shots_on_target',
       'home.yellow_cards_count', 'home.red_cards_count', 'home.fouls',
       'home.corners', 'home.crosses', 'home.touches', 'home.tackles',
       'home.interceptions', 'home.aerials_won', 'home.clearances',
       'home.offsides', 'home.goal_kicks', 'home.throw_ins', 'home.long_balls']
    guest_features = ['guest.score', 'guest.xG', 'guest.possession',
        'guest.passing_accuracy', 'guest.shots_on_target',
        'guest.yellow_cards_count', 'guest.red_cards_count', 'guest.fouls',
        'guest.corners', 'guest.crosses', 'guest.touches', 'guest.tackles',
        'guest.interceptions', 'guest.aerials_won', 'guest.clearances',
        'guest.offsides', 'guest.goal_kicks', 'guest.throw_ins',
        'guest.long_balls']
    home_mapper = {
        'home.score': 'score',
        'home.xG': 'xG',
        'home.possession': 'possession',
        'home.passing_accuracy':'passing_accuracy', 
        'home.shots_on_target': 'shots_on_target',
        'home.yellow_cards_count': 'yellow_cards_count', 
        'home.red_cards_count': 'red_cards_count', 
        'home.fouls': 'fouls',
        'home.corners': 'corners', 
        'home.crosses': 'crosses', 
        'home.touches': 'touches', 
        'home.tackles': 'tackles',
        'home.interceptions': 'interceptions',
        'home.aerials_won': 'aerials_won', 
        'home.clearances': 'clearances',
        'home.offsides': 'offsides', 
        'home.goal_kicks':'goal_kicks', 
        'home.throw_ins':'throw_ins',
        'home.long_balls':'long_balls'
    }
    guest_mapper = {
        'guest.score': 'score',
        'guest.xG': 'xG',
        'guest.possession': 'possession',
        'guest.passing_accuracy':'passing_accuracy', 
        'guest.shots_on_target': 'shots_on_target',
        'guest.yellow_cards_count': 'yellow_cards_count', 
        'guest.red_cards_count': 'red_cards_count', 
        'guest.fouls': 'fouls',
        'guest.corners': 'corners', 
        'guest.crosses': 'crosses', 
        'guest.touches': 'touches', 
        'guest.tackles': 'tackles',
        'guest.interceptions': 'interceptions',
        'guest.aerials_won': 'aerials_won', 
        'guest.clearances': 'clearances',
        'guest.offsides': 'offsides', 
        'guest.goal_kicks':'goal_kicks', 
        'guest.throw_ins':'throw_ins',
        'guest.long_balls':'long_balls'
    }
    for index, row in last_matches.iterrows():
       if team == row['home.team']:
            game = row[home_features].to_frame().transpose()
            game.rename(columns = home_mapper, inplace=True)
            result = pd.concat([result, game], axis=0)
       else:
            game = row[guest_features].to_frame().transpose()
            game.rename(columns = guest_mapper, inplace=True)
            result = pd.concat([result, game], axis = 0)
    return result.mean().to_frame().transpose()

In [13]:
def get_data(df: pd.DataFrame, n: int) -> pd.DataFrame:
   '''
      Функция, которая создает данные для машинного обучения: для каждого матча считает разницу в агрегированных признаках между домашней и гостевой командой

      Args:
         df: Датафрейм, по которому создаются данные
         n: Количество прошлых матчей, которое берется для агрегации 
      
      Returns: 
         data: Датафрейм с признаками по матчам
   '''
   df.sort_values(by = 'match_date', ascending=False, inplace=True)
   data = pd.DataFrame(columns=['score', 'xG', 'possession',
      'passing_accuracy', 'shots_on_target',
      'yellow_cards_count', 'red_cards_count', 'fouls',
      'corners', 'crosses', 'touches', 'tackles',
      'interceptions', 'aerials_won', 'clearances',
      'offsides', 'goal_kicks', 'throw_ins',
      'long_balls', 'head_to_head', 'result'])
   for index, row in df[:-n].iterrows():
      row = row.to_frame().transpose()
      home = get_aggregate(row['home.team'].values[0], n, df).reset_index(drop=True)
      guest = get_aggregate(row['guest.team'].values[0], n, df).reset_index(drop=True)
      data = pd.concat([data, pd.concat([home - guest, row[['head_to_head', 'result']].reset_index(drop=True)], axis=1)], axis = 0)
   return data.reset_index()

Соберем данные с 5 матчами в качестве гиперпараметра. В рамках анализа ещё пробовалось 2, 10 и 15 матчей, но качество модели лучше всего было при 5 матчах, поэтому возьмем его.

P.S. следующий блок работает не очень быстро, около 1.5 минут

In [16]:
data = get_data(new_df[new_df['match_date'] > '2017-05-21'], 5)

Разобьем выборку на тренировочкую и тестовую, размер тестовой возьмем 0.3 от выборки

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = data.drop('result', axis=1)
y = data['result'].astype(int)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)

Отмасштабируем признаки с помощью StandardScaler

In [20]:
from sklearn.preprocessing import StandardScaler

In [21]:
scaler = StandardScaler()
columns = X_train.columns
X_train = pd.DataFrame(data = scaler.fit_transform(X_train), columns=columns)
X_test = pd.DataFrame(data = scaler.transform(X_test), columns=columns)

В качестве модели возьмем логистическую регресиию для многоклассовой классификации. В данном блоке мы пробовали (код мы правда не добавили) и другие модели: Random Forest и GradientBoosting из sklearn, но логистическая регрессия показала лучшее результаты из всех  

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

logreg = LogisticRegression(random_state=15)
logreg.fit(X = X_train, y = y_train)

LogisticRegression(random_state=15)

In [23]:
accuracy_score(y_test, logreg.predict(X_test))

0.5203619909502263

In [24]:
roc_auc_score(y_test, logreg.predict_proba(X_test), multi_class='ovo')

0.6402524732609894